# Production of alpha-tocopherol (vitamin E) in E. coli

## 1. Introduction

### 1.1 Literature review of the compound (<500 words) Daniel

### 1.2 Literature review of the cell factory (<500 words) Ben

## 2. Problem definition (<300 words) Kaare

The goal of the project is to suggest an engineering strategy for the heterologous production of Vitamin E (tocochromanols/-trienols) based on the adaptation and predictions of a GSMM. The modelling decisions should be informed by the need of the bioprocess to be GMP-compatible, e.g. by the use of glucose as the main carbons souce. Currently, only precursors of this class of molecules of interest have been produced, in insufficient titers for real-world use, and at slow rates. We hope to be able to contribute suggestions for an engineering strategy that can improve the titer and productivity of the process.


## 3. *If Project category II:* Selection and assessment of existing GSM (<500 words) Joachim

## 4. Computer-Aided Cell Factory Engineering (<1500 words if Category II project; <500 words for Category I project)

## 5. Discussion (<500 words)

## 6. Conclusion (<200 words)

## References